In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
import sqlite3

# Connect to SQLite database (creates the database file if it doesn't exist)
conn = sqlite3.connect('books_inventory1.db')
cursor = conn.cursor()

# Create the Inventory table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Inventory (
    entry_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    genre TEXT NOT NULL,
    publication_date DATE,
    isbn TEXT UNIQUE NOT NULL
)
''')

conn.commit()
print("Database setup complete!")

Database setup complete!


# ADD NEW BOOKS

In [2]:
def add_book(title, author, genre, publication_date, isbn):
    try:
        cursor.execute('''
        INSERT INTO Inventory (title, author, genre, publication_date, isbn)
        VALUES (?, ?, ?, ?, ?)
        ''', (title, author, genre, publication_date, isbn))
        conn.commit()
        print("Book added successfully!")
    except sqlite3.IntegrityError as e:
        print(f"Error: {e}")

# Example Usage
add_book("The Great Gatsby", "F. Scott Fitzgerald", "Classic", "1925-04-10", "9780743273565")
add_book("1984", "George Orwell", "Dystopian", "1949-06-08", "9780451524935")

Book added successfully!
Book added successfully!


# FILTER BOOKS

In [3]:
import pandas as pd

def filter_books(title=None, author=None, genre=None, date_from=None, date_to=None):
    query = "SELECT * FROM Inventory WHERE 1=1"
    params = []

    if title:
        query += " AND title LIKE ?"
        params.append(f"%{title}%")
    if author:
        query += " AND author LIKE ?"
        params.append(f"%{author}%")
    if genre:
        query += " AND genre = ?"
        params.append(genre)
    if date_from:
        query += " AND publication_date >= ?"
        params.append(date_from)
    if date_to:
        query += " AND publication_date <= ?"
        params.append(date_to)
    
    result = pd.read_sql_query(query, conn, params=params)
    return result

# Example Usage
filtered_books = filter_books(author="George Orwell")
print(filtered_books)

   entry_id title         author      genre publication_date           isbn
0         2  1984  George Orwell  Dystopian       1949-06-08  9780451524935


# EXPORT DATA TO CSV

In [7]:
def export_to_csv(file_name="books_inventory1.csv"):
    books = pd.read_sql_query("SELECT * FROM Inventory", conn)
    books.to_csv(file_name, index=False)
    print(f"Data exported to {file_name}!")

# Example Usage
export_to_csv()

Data exported to books_inventory1.csv!


# EXPORT DATA TO JSON

In [12]:
import json
def export_to_json(file_name="books_inventory1.json"):
    books = pd.read_sql_query("SELECT * FROM Inventory", conn)
    books.to_json(file_name, orient='records')
    print(f"Data exported to {file_name}!")

# Example Usage
export_to_json()

Data exported to books_inventory1.json!


# TESTING THE CRUD OPERATIONS GIVEN

In [13]:
add_book("To Kill a Mockingbird", "Harper Lee", "Fiction", "1960-07-11", "9780061120084")
add_book("Brave New World", "Aldous Huxley", "Dystopian", "1932-01-01", "9780060850524")

Book added successfully!
Book added successfully!


In [14]:
print(filter_books(genre="Fiction"))

   entry_id                  title      author    genre publication_date  \
0         3  To Kill a Mockingbird  Harper Lee  Fiction       1960-07-11   

            isbn  
0  9780061120084  


In [15]:
export_to_csv()

Data exported to books_inventory1.csv!


In [16]:
export_to_json()

Data exported to books_inventory1.json!


# TO VERIFY THE SAVED DATABASE

In [1]:
import sqlite3

conn = sqlite3.connect('books_inventory1.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM Inventory")
rows = cursor.fetchall()

for row in rows:
    print(row)

(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 'Classic', '1925-04-10', '9780743273565')
(2, '1984', 'George Orwell', 'Dystopian', '1949-06-08', '9780451524935')
(3, 'To Kill a Mockingbird', 'Harper Lee', 'Fiction', '1960-07-11', '9780061120084')
(4, 'Brave New World', 'Aldous Huxley', 'Dystopian', '1932-01-01', '9780060850524')
